<h1 style="text-align: center;">
<b>Gastos Hospitalares</b>
</h1>

<h3 style="text-align: justify;">
Script para criação de mapas com dados de gastos hospitalares no Brasil.
</h3>

<p style='text-align: justify;'>
Os dados presentes nas análises foram extraídos do Tabnet. O Tabnet é uma 
ferramenta desenvolvida pelo Departamento de Informação e Informática do Sistema 
Único de Saúde (DataSUS) da Secretaria de Informação e Saúde Digital do Ministério
da Saúde (MS). Esta ferramenta abrange dados epidemiológicos, de mortalidade, 
da rede assistencial, de internações hospitalares, de procedimentos realizados 
na atenção primária, entre outras.
Para este trabalho, foram usadas as informações do Sistema de Informações 
Hospitalares (SIH), referentes às internações hospitalares aprovadas no período 
de 2014 a 2022. Foram usados os dados de Autorização de Internação Hospitalar 
(AIH) no formato reduzido (RD) com abrangência geográfica no nível municipal. 
Foi selecionado para o campo linha os municípios brasileiros. Para as colunas, 
foi selecionado o ano/mês de processamento.
</p>

<h4 style="text-align: justify;">
<b>Gastos Hospitalares</b>
<h4>

<p style="text-align: justify;">
O valor do gasto hospitalar representa o valor total dos procedimentos presentes na AIH realizados naquela internação. Esse valores são regulamentados pela Tabela Unificada de Procedimentos, Medicamentos e Órteses, Próteses e Materiais Especiais do SUS (SIGTAP).
</p>

<h4 style="text-align: justify;">
<b>Mortalidade</b>
<h4>

<p style="text-align: justify;">
A taxa de mortalidade é calculada pela razão entre o número de óbitos e número de internações, dada também pelo TabNet.
</p>

<h4 style="text-align: justify;">
<b>População</b>
<h4>
<p style='text-align: justify;'>
Os dados de população dos municípios utilizados neste trabalho, foram obtidas a partirde duas fontes. Para os anos de 2014 a 2021, foram utilizadas as estimativas preliminares
elaboradas pelo Ministério da Saúde, disponibilizados pelo TabNet.
</p>
<p style='text-align: justify;'>
O dado de população em 2022 foi obtido no site do Instituto Brasileiro de Geografia eEstatística - IBGE.
</p>

<p style="text-align: justify;">
<b>Acesse os dados no portal</b>

[Datasus Saúde Governo Federal](https://datasus.saude.gov.brinformacoes-de-saude-tabnet/)
</p>

<p style="text-align: justify;">

[Panorama Censo 2022 - IBGE](https://censo2022.ibge.gov.br/panorama/)
</p>

<p style="text-align: justify;">
___________________________________________


In [ ]:
import geobr
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [ ]:
df = pd.read_excel('base_tratada.xlsx', engine='openpyxl')

df.head()

In [ ]:
# criando um novo df com ano de 2022 


df_2022 = df[df['ano'] == 2022].reset_index(drop=True)

df_2022['cod_ibge'] = df_2022['cod_ibge'].apply(lambda x: str(x))
df_2022.head()

In [ ]:
df_2022.info()

In [ ]:
# importando a malha municipal do geobr

muni = geobr.read_municipality(year=2020)
muni

In [ ]:
# ajeitando a variável de código municipal para ser compativel com a base de dados
# deixando código municípal com 6 dígitos

muni['cod_municipal'] = muni['code_muni'].apply(lambda x: str(int(x)))

muni['cod_municipal'] = muni['cod_municipal'].str[:-1]

muni.head()

In [ ]:
#juntando a malha com a base

muni = muni.merge(df_2022, how="left", left_on="cod_municipal", right_on="cod_ibge")
muni.head()

In [ ]:
# plotando mapa dos gastos hospitalares em 2022


plt.rcParams.update({"font.size": 5})

fig, ax = plt.subplots(figsize=(4, 4), dpi=200)

muni.plot(
    column="gastos",
    cmap="RdYlGn_r",
    legend=True,
    ax=ax,
)

ax.set_title("Gastos Hospitalares no Brasil em 2022")
ax.axis("off")

In [ ]:
# plotando mapa da taxa de mortalidade em 2022


plt.rcParams.update({"font.size": 5})

fig, ax = plt.subplots(figsize=(4, 4), dpi=200)

muni.plot(
    column="mortalidade",
    cmap="RdYlGn_r",
    legend=True,
    ax=ax,
)

ax.set_title("Mortalidade Hospitalar em 2022")
ax.axis("off")

In [ ]:
# agrupando dados por uf e ano

uf_22 = df_2022.groupby('uf').agg({'gastos': 'sum', 'mortalidade': 'mean', 
                                           'populacao': 'sum'}).reset_index()

uf_22.info()

In [ ]:
states = geobr.read_state(year=2020)
states = states.merge(uf_22, how="left", left_on="abbrev_state", right_on="uf")
states

In [ ]:

# plotando mapa dos gastos por uf


plt.rcParams.update({"font.size": 5})

fig, ax = plt.subplots(figsize=(4, 4), dpi=200)

states.plot(
    column="gastos",
    cmap="RdYlGn_r",
    legend=True,
    ax=ax,
)

ax.set_title("Gastos Hospitalares no Brasil em 2022")
ax.axis("off")

In [ ]:

# plotando mapa da mortalidade


plt.rcParams.update({"font.size": 5})

fig, ax = plt.subplots(figsize=(4, 4), dpi=200)

states.plot(
    column="mortalidade",
    cmap="RdYlGn_r",
    legend=True,
    ax=ax,
)

ax.set_title("Mortalidade Hospitalar em 2022")
ax.axis("off")